In [2]:
import pandas as pd
import numpy as np

## Read in the Data and Find the Most Common EPA Site Type for Each County

In [3]:
# Read in data on mandatory EPA reporting sites
# https://www.epa.gov/frs/geospatial-data-download-service
epa = pd.read_csv('NATIONAL_SINGLE.CSV')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (4,5,7,11,13,14,15,16,21,24,27,28,29,30) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Visualize the first few rows of data
epa.head()

,FRS_FACILITY_DETAIL_REPORT_URL,REGISTRY_ID,PRIMARY_NAME,LOCATION_ADDRESS,SUPPLEMENTAL_LOCATION,CITY_NAME,COUNTY_NAME,FIPS_CODE,STATE_CODE,STATE_NAME,...,SIC_CODES,SIC_CODE_DESCRIPTIONS,LATITUDE83,LONGITUDE83,CONVEYOR,COLLECT_DESC,ACCURACY_VALUE,REF_POINT_DESC,HDATUM_DESC,SOURCE_DESC
0,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110058360857,FORT JENSEN,NaN,NaN,1,BRISTOL BAY,02060,AK,ALASKA,...,NaN,NaN,57.885100,-157.096820,EIS,NaN,NaN,NaN,NAD83,NaN
1,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110058283236,ADAK COD COOPERATIVE LLC,UNKNOWN,NaN,ADAK,ALEUTIANS WEST,NaN,AK,ALASKA,...,2091,CANNED AND CURED FISH AND SEAFOODS,51.862620,-176.637235,ICIS,GPS - UNSPECIFIED,NaN,NaN,NAD83,NaN
2,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110038070002,ADAK NAVAL AIR STATION SITE,MAIN RD,ADAK ISLAND,ADAK,ALEUTIANS WEST,02016,AK,ALASKA,...,4952,SEWERAGE SYSTEMS,51.877500,-176.652500,SEMS,NaN,NaN,NaN,NAD83,NaN
3,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110070215939,ADAK PETROLEUM LLC BULK FUEL FACILITY,10252 TRANSIT ROAD,NaN,ADAK,ALEUTIANS WEST (CA),AK016,AK,ALASKA,...,5171,PETROLEUM BULK STATIONS AND TERMINALS,51.863056,-176.639444,ICIS,NaN,NaN,NaN,NAD83,NaN
4,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110015787683,ADAK POWER PLANT,100 HILLSIDE BLVD,NaN,ADAK,ALEUTIANS WEST,02016,AK,ALASKA,...,"9711, OWNE",NATIONAL SECURITY,51.870300,-176.659000,EIS,UNKNOWN,NaN,NaN,NAD83,NaN


In [5]:
# Below I am having problems with county names not being recognized as the same, I think extra spaces are the reason
# so i am using the strip function to remove spaces
epa['STATE_CODE'] = epa['STATE_CODE'].str.strip()
epa['COUNTY_NAME'] = epa['COUNTY_NAME'].str.strip()
epa['SIC_CODE_DESCRIPTIONS'] = epa['SIC_CODE_DESCRIPTIONS'].str.strip()


In [6]:
# Make all columns visable
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Make a new column that will act as a counter
epa['counter']=1


In [7]:
# Make a copy of the epa data frame
epa_sic = epa

In [8]:
# Drop all nan values in the SIC_CODES column
epa_sic['SIC_CODE_DESCRIPTIONS'].dropna(inplace=True)

In [9]:
# A single site might have multiple LST codes and right now they are a string separated by commas, to do the 
# analysis they need to be in a list and then we can separate them out into a row for each SIC code, that way 
# we can count how many of each SIC code are in each county and get a sense of what the top EPA concerns are in each area
# i.e. mining, manufacturing etc. 
# I used this resource for this code chunk https://stackoverflow.com/questions/12680754/split-explode-pandas-dataframe-string-entry-to-separate-rows

lst_col = 'SIC_CODE_DESCRIPTIONS' 

epa_sic = epa_sic.assign(**{lst_col:epa_sic[lst_col].str.split(',')})

epa_sic.head()

,FRS_FACILITY_DETAIL_REPORT_URL,REGISTRY_ID,PRIMARY_NAME,LOCATION_ADDRESS,SUPPLEMENTAL_LOCATION,CITY_NAME,COUNTY_NAME,FIPS_CODE,STATE_CODE,STATE_NAME,COUNTRY_NAME,POSTAL_CODE,FEDERAL_FACILITY_CODE,FEDERAL_AGENCY_NAME,TRIBAL_LAND_CODE,TRIBAL_LAND_NAME,CONGRESSIONAL_DIST_NUM,CENSUS_BLOCK_CODE,HUC_CODE,EPA_REGION_CODE,SITE_TYPE_NAME,LOCATION_DESCRIPTION,CREATE_DATE,UPDATE_DATE,US_MEXICO_BORDER_IND,PGM_SYS_ACRNMS,INTEREST_TYPES,NAICS_CODES,NAICS_CODE_DESCRIPTIONS,SIC_CODES,SIC_CODE_DESCRIPTIONS,LATITUDE83,LONGITUDE83,CONVEYOR,COLLECT_DESC,ACCURACY_VALUE,REF_POINT_DESC,HDATUM_DESC,SOURCE_DESC,counter
0,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110058360857,FORT JENSEN,NaN,NaN,1,BRISTOL BAY,02060,AK,ALASKA,UNITED STATES,00000,NaN,NaN,NaN,NaN,00,2.164000e+13,19030202.0,10.0,STATIONARY,NaN,06-APR-14,01-JUN-17,NaN,EIS:16091811,AIR EMISSIONS CLASSIFICATION UNKNOWN,NaN,NaN,NaN,NaN,57.885100,-157.096820,EIS,NaN,NaN,NaN,NAD83,NaN,1
1,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110058283236,ADAK COD COOPERATIVE LLC,UNKNOWN,NaN,ADAK,ALEUTIANS WEST,NaN,AK,ALASKA,NaN,99546,NaN,NaN,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,05-APR-14,11-JAN-16,NaN,NPDES:AKU000292,ICIS-NPDES UNPERMITTED,NaN,NaN,2091,[CANNED AND CURED FISH AND SEAFOODS],51.862620,-176.637235,ICIS,GPS - UNSPECIFIED,NaN,NaN,NAD83,NaN,1
2,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110038070002,ADAK NAVAL AIR STATION SITE,MAIN RD,ADAK ISLAND,ADAK,ALEUTIANS WEST,02016,AK,ALASKA,UNITED STATES,99546,Yes,Defense: Navy,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,18-FEB-09,01-MAR-19,NaN,"BRAC:AK4170024323, EIS:10606011, FFDOCKET:AK41...","AIR EMISSIONS CLASSIFICATION UNKNOWN, BRAC, CE...",NaN,NaN,4952,[SEWERAGE SYSTEMS],51.877500,-176.652500,SEMS,NaN,NaN,NaN,NAD83,NaN,1
3,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110070215939,ADAK PETROLEUM LLC BULK FUEL FACILITY,10252 TRANSIT ROAD,NaN,ADAK,ALEUTIANS WEST (CA),AK016,AK,ALASKA,NaN,99546,NaN,NaN,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,06-APR-18,NaN,NaN,NPDES:AKR06AA03,"ICIS-NPDES NON-MAJOR, STORM WATER INDUSTRIAL",NaN,NaN,5171,[PETROLEUM BULK STATIONS AND TERMINALS],51.863056,-176.639444,ICIS,NaN,NaN,NaN,NAD83,NaN,1
4,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110015787683,ADAK POWER PLANT,100 HILLSIDE BLVD,NaN,ADAK,ALEUTIANS WEST,02016,AK,ALASKA,UNITED STATES,99546,NaN,NaN,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,20-NOV-03,01-JUN-17,NaN,"AIR:AK0000000201600016, AIRS/AFS:0201600016, E...","AIR EMISSIONS CLASSIFICATION UNKNOWN, AIR MAJOR",928110,NATIONAL SECURITY.,"9711, OWNE",[NATIONAL SECURITY],51.870300,-176.659000,EIS,UNKNOWN,NaN,NaN,NAD83,NaN,1


In [10]:
# Right now each site has a list of SIC codes which indicate what type of site it is, I want to make it one row per
# SIC code, so there might be 3 rows for a site with 3 different uses, I am using the pd.explode function for this
epa_sic = pd.DataFrame(epa_sic)
epa_sic = epa_sic.explode('SIC_CODE_DESCRIPTIONS')





In [11]:
# We have to drop nans again since they were sometimes in a list before and are now in their own cells
epa_sic['SIC_CODE_DESCRIPTIONS'].dropna(inplace=True)




In [12]:
epa_sic.head()


,FRS_FACILITY_DETAIL_REPORT_URL,REGISTRY_ID,PRIMARY_NAME,LOCATION_ADDRESS,SUPPLEMENTAL_LOCATION,CITY_NAME,COUNTY_NAME,FIPS_CODE,STATE_CODE,STATE_NAME,COUNTRY_NAME,POSTAL_CODE,FEDERAL_FACILITY_CODE,FEDERAL_AGENCY_NAME,TRIBAL_LAND_CODE,TRIBAL_LAND_NAME,CONGRESSIONAL_DIST_NUM,CENSUS_BLOCK_CODE,HUC_CODE,EPA_REGION_CODE,SITE_TYPE_NAME,LOCATION_DESCRIPTION,CREATE_DATE,UPDATE_DATE,US_MEXICO_BORDER_IND,PGM_SYS_ACRNMS,INTEREST_TYPES,NAICS_CODES,NAICS_CODE_DESCRIPTIONS,SIC_CODES,SIC_CODE_DESCRIPTIONS,LATITUDE83,LONGITUDE83,CONVEYOR,COLLECT_DESC,ACCURACY_VALUE,REF_POINT_DESC,HDATUM_DESC,SOURCE_DESC,counter
0,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110058360857,FORT JENSEN,NaN,NaN,1,BRISTOL BAY,02060,AK,ALASKA,UNITED STATES,00000,NaN,NaN,NaN,NaN,00,2.164000e+13,19030202.0,10.0,STATIONARY,NaN,06-APR-14,01-JUN-17,NaN,EIS:16091811,AIR EMISSIONS CLASSIFICATION UNKNOWN,NaN,NaN,NaN,NaN,57.885100,-157.096820,EIS,NaN,NaN,NaN,NAD83,NaN,1
1,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110058283236,ADAK COD COOPERATIVE LLC,UNKNOWN,NaN,ADAK,ALEUTIANS WEST,NaN,AK,ALASKA,NaN,99546,NaN,NaN,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,05-APR-14,11-JAN-16,NaN,NPDES:AKU000292,ICIS-NPDES UNPERMITTED,NaN,NaN,2091,CANNED AND CURED FISH AND SEAFOODS,51.862620,-176.637235,ICIS,GPS - UNSPECIFIED,NaN,NaN,NAD83,NaN,1
2,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110038070002,ADAK NAVAL AIR STATION SITE,MAIN RD,ADAK ISLAND,ADAK,ALEUTIANS WEST,02016,AK,ALASKA,UNITED STATES,99546,Yes,Defense: Navy,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,18-FEB-09,01-MAR-19,NaN,"BRAC:AK4170024323, EIS:10606011, FFDOCKET:AK41...","AIR EMISSIONS CLASSIFICATION UNKNOWN, BRAC, CE...",NaN,NaN,4952,SEWERAGE SYSTEMS,51.877500,-176.652500,SEMS,NaN,NaN,NaN,NAD83,NaN,1
3,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110070215939,ADAK PETROLEUM LLC BULK FUEL FACILITY,10252 TRANSIT ROAD,NaN,ADAK,ALEUTIANS WEST (CA),AK016,AK,ALASKA,NaN,99546,NaN,NaN,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,06-APR-18,NaN,NaN,NPDES:AKR06AA03,"ICIS-NPDES NON-MAJOR, STORM WATER INDUSTRIAL",NaN,NaN,5171,PETROLEUM BULK STATIONS AND TERMINALS,51.863056,-176.639444,ICIS,NaN,NaN,NaN,NAD83,NaN,1
4,https://ofmpub.epa.gov/frs_public2/fii_query_d...,110015787683,ADAK POWER PLANT,100 HILLSIDE BLVD,NaN,ADAK,ALEUTIANS WEST,02016,AK,ALASKA,UNITED STATES,99546,NaN,NaN,NaN,NaN,00,2.016000e+13,19030103.0,10.0,STATIONARY,NaN,20-NOV-03,01-JUN-17,NaN,"AIR:AK0000000201600016, AIRS/AFS:0201600016, E...","AIR EMISSIONS CLASSIFICATION UNKNOWN, AIR MAJOR",928110,NATIONAL SECURITY.,"9711, OWNE",NATIONAL SECURITY,51.870300,-176.659000,EIS,UNKNOWN,NaN,NaN,NAD83,NaN,1


In [13]:
# Each column will contain the count of that site type for each county
sic_numbers = pd.pivot_table(epa_sic, values='counter',index=['STATE_CODE','COUNTY_NAME'],columns= 'SIC_CODE_DESCRIPTIONS',aggfunc=np.sum)

In [14]:
sic_numbers.shape

(6163, 2277)

In [15]:
# Examining the columns, there
cols = sic_numbers.columns.tolist()
#I printed the columns to get the names exactly right but commented out so they won't all print here
#cols

In [16]:
# I don't want 'NOT ELSEWHERE CLASSIFIED'  or 'NONCLASSIFIABLE ESTABLISHMENTS' to come up as the top 
# site type for any county so I am removing them

sic_numbers = sic_numbers.drop([' NOT ELSEWHERE CLASSIFIED',' NONCLASSIFIABLE ESTABLISHMENTS','NONCLASSIFIABLE ESTABLISHMENTS'], axis =1)


sic_numbers.head()

SIC_CODE_DESCRIPTIONS                   ACOUSTICAL   ACTORS  \
STATE_CODE COUNTY_NAME                                        
AK         ALEUTIANS EAST                      NaN      NaN   
           ALEUTIANS EAST (B)                  NaN      NaN   
           ALEUTIANS WEST                      NaN      NaN   
           ALEUTIANS WEST (CA)                 NaN      NaN   
           ALEUTIANS WEST CENSUS AREA          NaN      NaN   

SIC_CODE_DESCRIPTIONS                   ADHESIVES AND SEALANTS  \
STATE_CODE COUNTY_NAME                                           
AK         ALEUTIANS EAST                                  NaN   
           ALEUTIANS EAST (B)                              NaN   
           ALEUTIANS WEST                                  NaN   
           ALEUTIANS WEST (CA)                             NaN   
           ALEUTIANS WEST CENSUS AREA                      NaN   

SIC_CODE_DESCRIPTIONS                   ADJUSTMENT AND COLLECTION SERVICES  \
STATE_CODE COUNTY_NAME                                                       
AK         ALEUTIANS EAST                                              NaN   
           ALEUTIANS EAST (B)                                          NaN   
           ALEUTIANS WEST                                              NaN   
           ALEUTIANS WEST (CA)                                         NaN   
           ALEUTIANS WEST CENSUS AREA                                  NaN   

SIC_CODE_DESCRIPTIONS                   ADMINISTRATION OF PUBLIC HEALTH PROGRAMS  \
STATE_CODE COUNTY_NAME                                                             
AK         ALEUTIANS EAST                                                    NaN   
           ALEUTIANS EAST (B)                                                NaN   
           ALEUTIANS WEST                                                    NaN   
           ALEUTIANS WEST (CA)                                               NaN   
           ALEUTIANS WEST CENSUS AREA                                        NaN   

SIC_CODE_DESCRIPTIONS                   ADMINISTRATION OF SOCIAL  \
STATE_CODE COUNTY_NAME                                             
AK         ALEUTIANS EAST                                    NaN   
           ALEUTIANS EAST (B)                                NaN   
           ALEUTIANS WEST                                    NaN   
           ALEUTIANS WEST (CA)                               NaN   
           ALEUTIANS WEST CENSUS AREA                        NaN   

SIC_CODE_DESCRIPTIONS                   ADMINISTRATION OF URBAN PLANNING AND COMMUNITY AND RURAL DEVELOPMENT  \
STATE_CODE COUNTY_NAME                                                                                         
AK         ALEUTIANS EAST                                                            NaN                       
           ALEUTIANS EAST (B)                                                        NaN                       
           ALEUTIANS WEST                                                            NaN                       
           ALEUTIANS WEST (CA)                                                       NaN                       
           ALEUTIANS WEST CENSUS AREA                                                NaN                       

SIC_CODE_DESCRIPTIONS                   ADVERTISING   ADVERTISING AGENCIES  \
STATE_CODE COUNTY_NAME                                                       
AK         ALEUTIANS EAST                       NaN                    NaN   
           ALEUTIANS EAST (B)                   NaN                    NaN   
           ALEUTIANS WEST                       NaN                    NaN   
           ALEUTIANS WEST (CA)                  NaN                    NaN   
           ALEUTIANS WEST CENSUS AREA           NaN                    NaN   

SIC_CODE_DESCRIPTIONS                   AERONAUTICAL  \
STATE_CODE COUNTY_NAME                                 
AK         ALEUTIANS EAST             

In [17]:
sic_numbers.shape

(6163, 2274)

In [18]:
# Now we want to find which column has the highest value for each county that is what is the most common type of site
# in each county

top_site_type = pd.DataFrame(sic_numbers.idxmax(axis=1))

In [20]:
top_site_type.head()

0
STATE_CODE COUNTY_NAME                                                            
AK         ALEUTIANS EAST               PREPARED FRESH OR FROZEN FISH AND SEAFOODS
           ALEUTIANS EAST (B)           PREPARED FRESH OR FROZEN FISH AND SEAFOODS
           ALEUTIANS WEST               PREPARED FRESH OR FROZEN FISH AND SEAFOODS
           ALEUTIANS WEST (CA)                  CANNED AND CURED FISH AND SEAFOODS
           ALEUTIANS WEST CENSUS AREA   PREPARED FRESH OR FROZEN FISH AND SEAFOODS

In [22]:
# Reset the index so the merge below works properly
top_site_type.reset_index(inplace=True)

## Find Out How Many EPA Reporting Sites There are In Each County

In [23]:
# I want to get a dataframe that is just the name of the county and the number of sites in that county
# I will use pivot tables
county_numbers = pd.pivot_table(epa, values='counter',index=['STATE_CODE','COUNTY_NAME'],aggfunc=np.sum)

In [24]:
county_numbers.head()

counter
STATE_CODE COUNTY_NAME                    
AK         ALASKA                        1
           ALEUTIAN ISLANDS              5
           ALEUTIANS EAST               62
           ALEUTIANS EAST (B)            1
           ALEUTIANS EAST BOROUGH        3

In [25]:
# Lets get a look at what the numbers are like

county_numbers.describe()

,counter
count,7994.000000
mean,474.232299
std,2058.777070
min,1.000000
25%,4.000000
50%,39.000000
75%,333.000000
max,98458.000000


## Join the Tables So Each County Has a Count and Top Site Type

In [26]:
# Check the shape
county_numbers.shape

(7994, 1)

In [27]:
# Check the shape
top_site_type.shape

(6163, 4)

In [28]:
# Looks like there is not a top site type for every county, so we wan to do a left join so that missing ones will
# be filled in with NaN and we won't lose an county data
result = county_numbers.merge(top_site_type, how='left', on=['STATE_CODE','COUNTY_NAME'])


In [29]:
# Check that there are 7994 rows and not 6163 to make sure that the merge worked as expected
result.shape

(7994, 5)

In [31]:
result.head()

,STATE_CODE,COUNTY_NAME,counter,index,0
0,AK,ALASKA,1,NaN,NaN
1,AK,ALEUTIAN ISLANDS,5,NaN,NaN
2,AK,ALEUTIANS EAST,62,0.0,PREPARED FRESH OR FROZEN FISH AND SEAFOODS
3,AK,ALEUTIANS EAST (B),1,1.0,PREPARED FRESH OR FROZEN FISH AND SEAFOODS
4,AK,ALEUTIANS EAST BOROUGH,3,NaN,NaN


In [65]:
# Export the results to a csv that can be imported into Tableau
result.to_csv('df.csv')